# colab版MMD自動トレースへようこそ！(実行編)

# 始めに

このツールの稼働状況やメンテナンス情報はTwitter（[@miu200521358](https://twitter.com/miu200521358)）にて行っています。

エラーになる、起動しない、などの場合、まずは現在の配布状況をご確認ください。

リプやDM等でのお問い合わせも受け付けています。

# MMD自動トレースキット準備

このノートブックでは、MMD自動トレースの準備と実行を行います。


画面左上の「＞」をクリックして下さい。目次が開きます。（既に開いている場合は次へ進んでください）

![目次](https://drive.google.com/uc?export=view&id=1x8AdFNmsIQPrtYptBf_NXPRNBJF8ON8z)


ノートブックを上から順に確認し、以下手順をひとつずつ実行してください。

実行が必要なセルには番号を振ってあります。①から順番に実行してください。

※　準備一括実行だけは番号が振れないので、③が抜けています。

- **「①　環境設定」**
  - ランタイムがGPUに変更できたことを確認します
    - 変更のやり方は、導入編をご確認ください
  - Tensorflowのバージョンを1.xに変更します
  - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロードします
   - 準備や実際のトレース処理等、長い処理時に鳴らします
   - 不要の場合は、ブラウザの音量をミュートにしてください
- **「②　Googleドライブとの連携」**
  - Googleドライブとの連携ができたことを確認します
  - 連携のやり方は、導入編をご確認ください
- **「③　準備一括実行」**
    - 準備セクションのセルをすべて実行します
      - この処理で、MMD自動トレースに必要なプログラムやデータがすべてcolab上に作成されます。
      - 大体40～60分くらいかかります。
- **「④　MMD自動トレースキット実行」**
  - 実行セクションのセルを上から順番に1つずつ実行します
    - トレース元動画の指定
    - トレースパラメーターの設定
    - トレース処理実行
    - 人数にもよりますが、6000Fで大体50～60分くらいかかります。


## 環境設定

ヘッダの "ランタイム"　＞　"ランタイムのタイプを変更"　＞　"GPU"　を選択して下さい。

変更できたら、下のセルを実行して下さい。

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル①】　
#@markdown - ランタイムがGPUであることの確認
#@markdown - Tensorflowのバージョンを 1.x に変更
#@markdown - 効果音を[効果音ラボ](https://soundeffect-lab.info/)様よりダウンロード

#@markdown ■■■■■■■■■■■■■■■■■■

! echo --------------
! echo 【A】 ランタイムをGPUに変更
! echo --------------

! nvcc --version
! nvidia-smi

! echo --------------
! echo 【B】 Tensorflowのバージョンを 1.x に変更
! echo --------------

%tensorflow_version 1.x

%tensorflow_version

! echo --------------
! echo 【C】 効果音をダウンロード
! echo --------------

! wget -c "https://soundeffect-lab.info/sound/anime/mp3/sceneswitch1.mp3"

from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)

**【OK】**

下記のように表示されて、最後に音が鳴ったら成功です。

![GPU変更成功](https://drive.google.com/uc?export=view&id=1Iw-WeUzbN48jra98rHx9kYBcw9sHJwVe)

---

**【NG】**

下記のように表示されていたら、ランタイムの変更に失敗しているので、導入編を再度確認して、ランタイムを変更してください。

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1tufSuT7ocWxv3HkrmA5kwlemhu0gv6Je)

## Googleドライブ・GoogleSDKとの連携

Googleドライブの `autotrace` フォルダと連携します。

下のセルを実行してください。

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル②】　Googleドライブとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

**【OK】**

下記のように、「autotrace」フォルダの中身が表示されていたら成功です。

![Googleドライブ連携](https://drive.google.com/uc?export=view&id=19OhLvcyP-CN90KWDmkmBuSRR3BpO5GzV)

miuのGoogleドライブから必要なデータをDLするため、`GoogleSDK` と連携します。

セル②と同じく、URLからアクセス許可を与えてください。

下のセルを実行してください。

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル③】　GoogleSDKとの連携

#@markdown ■■■■■■■■■■■■■■■■■■

# Googleドライブアクセスライブラリ
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

! echo Google SDK との連携が成功しました。

## 準備一括実行

ここでは、準備セクションのセルを一括で実行します。

以下の手順を確認・実行して、準備一括実行セクションのセルをすべて実行してください。

---

目次の「MMD自動トレースキット実行」を選択して下さい。

![実行選択](https://drive.google.com/uc?export=view&id=1YaSrtio06mmu94RUkU5dSolNtFqPiAnV)

ヘッダの　"ランタイム"　＞　"より前のセルを実行"　を選択すると、準備セクションのすべてのセルが順次実行されます。

最初に音が鳴りますが、処理が開始された音なので、同じ音が鳴るまで待って下さい。

![すべてのセルを実行](https://drive.google.com/uc?export=view&id=1rIbVI_Qyjs8idEzkIR0IBIx6Dbp3te08)

---

** 【OK】**

画面の一番下に、以下のように出力されれば、完了です。

![処理成功](https://drive.google.com/uc?export=view&id=1D21xezv6QN0RQF5ZU_LR7PRnOk0Dw4Sc)

大体40～60分くらいかかります。

---

**【NG】**

![処理失敗](https://drive.google.com/uc?export=view&id=1t-immeF3Ji1_GBNatZOG1C07j42de4Rq)

最後の行に、「No such file or directory」と出力されていたら失敗です。

解決方法が分からない場合、ノートブックを共有してください。

### 環境設定

In [0]:
# 処理時間計測のための開始時間
import time
start_time = time.time()

In [0]:
# Openposeバージョンタグ
ver_openpose = "v1.5.1"

In [0]:
# MMD自動トレースキットバージョンタグ
ver_tag = "ver1.03.02"

### cmake

In [0]:
! cmake --version

In [0]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.17.2/cmake-3.17.2.tar.gz"
! tar xf cmake-3.17.2.tar.gz
! cd cmake-3.17.2 && ./configure && make && sudo make install

### Openpose

In [0]:
# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [0]:
#  Openpose の clone
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 
# ! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git     

In [0]:
# OpenposeモデルデータDL
! rm -r ./openpose/models

# Openposeモデルデータ の導入
! mkdir -p ./openpose/models

# Openposeモデルデータのダウンロード
downloaded = drive.CreateFile({'id': '1-XlC0b00ueiS8-fDqKeS8w5NQ35DOnFN'})
downloaded.GetContentFile('opnepose-models.zip')

! unzip ./opnepose-models.zip
! mv ./models ./openpose

In [0]:
# Create build directory
! cd openpose && mkdir build && cd build

In [0]:
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON
! cd openpose/build && cmake .. 

In [0]:
# Openpose BUilding
! cd openpose/build && make -j`nproc`

### mannequinchallenge-vmd

In [0]:
# mannequinchallenge-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/mannequinchallenge-vmd.git

In [0]:
# mannequinchallenge-vmd の モデルデータDL

# モデルデータのダウンロード
! cd  ./mannequinchallenge-vmd && ./fetch_checkpoints.sh

### 3d-pose-baseline-vmd

In [0]:
# 3d-pose-baseline-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/3d-pose-baseline-vmd.git

In [0]:
# 3d-pose-baseline-vmd の Human3.6MモデルデータDL

# Human3.6Mモデルデータ の導入
! mkdir -p ./3d-pose-baseline-vmd/data/h36m

# Human3.6Mモデルデータのダウンロード
downloaded = drive.CreateFile({'id': '1W5WoWpCcJvGm4CHoUhfIB0dgXBDCEHHq'})
downloaded.GetContentFile('h36m.zip')

! unzip ./h36m.zip

! mv ./h36m ./3d-pose-baseline-vmd/data/

In [0]:
# 3d-pose-baseline-vmd の 学習データDL

# 3d-pose-baseline用学習データ の導入
! mkdir -p ./3d-pose-baseline-vmd/experiments

# Human3.6Mモデルデータのダウンロード
downloaded = drive.CreateFile({'id': '1v7ccpms3ZR8ExWWwVfcSpjMsGscDYH7_'})
downloaded.GetContentFile('experiments.zip')

! unzip ./experiments.zip

! mv ./experiments ./3d-pose-baseline-vmd/

### VMD-3d-pose-baseline-multi

In [0]:
# VMD-3d-pose-baseline-multi の clone

! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/VMD-3d-pose-baseline-multi.git

In [0]:
# VMD-3d-pose-baseline-multi のライブラリ

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools

### 準備結果確認

In [0]:
# サンプルの実行確認
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json ./output/ --display 0  --write_video ./output/openpose.avi --model_pose COCO

In [0]:
import time

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo "Openpose実行結果"

! ls -l ./openpose/output/openpose.avi

from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)

# MMD自動トレースキット実行

## インストール完了確認

ここからは、キットを実際に実行していきます。

「より前のセルを実行する」で、インストールはすべて実行されましたでしょうか。

詳しくは、「準備開始」のセクションを確認してください。

準備完了されましたら、下のセルを実行して、インストールがすべて完了しているか確認してください。



In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑤】　インストール完了確認

#@markdown ※④は準備一括実行なので項番抜けています。

#@markdown ■■■■■■■■■■■■■■■■■■


!ls -l ./openpose/README.md
!ls -l ./mannequinchallenge-vmd/README.md
!ls -l ./3d-pose-baseline-vmd/README.md
!ls -l ./VMD-3d-pose-baseline-multi/README.md
!ls -l ./openpose/output/openpose.avi

**【OK】**

以下のように、ファイル名とファイルサイズが表示されていれば、インストール完了です。(日時は米時間のようです)

入力映像ファイルアップロードに進んでください。

![インストール成功](https://drive.google.com/uc?export=view&id=1l13A2iF9oTpGcZSe9q8k7JyDyiABxOQT)

---

**【NG】**

以下のように、「No such file or directory」と表示されている場合、インストール失敗です。

![インストール失敗](https://drive.google.com/uc?export=view&id=1LuKoSMwFOzFg8NguFxqAtmQy9B1_KMXr)

このノートブックの先頭に戻って初めから実行し直してください。

３回やってもインストールに失敗する場合、ノートブックを共有してください。

## 入力映像ファイルアップロード

処理したい映像ファイルを、準備してください。

 - ファイル名は **半角英数字のみ** にしてください。opencvは2バイト文字を読み込めません。
 - Googleドライブの **autotrace** フォルダ 直下に置いてください。
 - FPSは、**30fps** もしくは **60fps** にしてください。
 - 大きさは、**1280x720** にしてください。
 - 大きさもしくは fps が指定通りではない場合、プログラム側で再エンコードします。（fpsは30になります）
 - **マウント後のGooleドライブ上のファイルの上書きや更新は正しく認識されません。** 新しいファイルは新規の名前でアップロードしてから処理して下さい。
 - アップロードが完了したら、下のセルを順次実行して下さい。

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑥】　入力映像ファイルアップロード

#@markdown ■■■■■■■■■■■■■■■■■■


#@markdown ### 【O】入力映像ファイル
#@markdown 解析対象となる映像のファイルの名前を入力して下さい。

#@markdown 横幅が1280ではない、もしくは30fpsではない場合、再エンコードします。

input_video_name = "input.mp4"  #@param {type: "string"}

from google.colab import drive
import sys
import os
import cv2
import datetime
import math
from tqdm import tqdm
from PIL import Image
import traceback
import numpy as np

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

# 入力動画ファイル
input_video = base_path + "/"+ input_video_name

print("ファイル名: ", os.path.basename(input_video))
print("ファイルサイズ: ", os.path.getsize(input_video))


video = cv2.VideoCapture(input_video)
# 幅
W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
# 高さ
H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
# 総フレーム数
count = video.get(cv2.CAP_PROP_FRAME_COUNT)
# fps
fps = video.get(cv2.CAP_PROP_FPS)

print("横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))



if W >= H:
    # 横長の場合
    width = 1280
    height = 720
else:
    # 縦長の場合
    height = 1280
    width = 720

# 画面比率
aspect = width / height

if (aspect != (9/6) and aspect != (16/9)) or W != 1280 or H != 720 or (fps != 30 and fps != 60):
    print("大きさもしくはfpsが処理対象外のため、再エンコードします: "+ input_video)
    
    # 補間png出力先
    interpolation_output_path = "./interpolation.png"

    # 縮尺
    scale = width / W

    # オリジナル高さ
    im_height = int(H * scale)

    # 出力ファイルパス
    out_name = 'recode_{0}.mp4'.format("{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now()))
    out_path = '{0}/{1}'.format(base_path, out_name)
    cnt = 0

    try:
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        out = cv2.VideoWriter(out_path, fourcc, 30.0, (width, height))
        # 入力ファイル
        cap = cv2.VideoCapture(input_video)
        # オリジナル出力フレーム
        output_frames = []

        print ("元動画読み込み開始")

        for _ in range(int(count)):
        # for _ in tqdm(range(int(count))):
            if not cap.isOpened():
                break

            # 動画から1枚キャプチャして読み込む
            flag, frame = cap.read()  # Capture frame-by-frame

            # 動画が終わっていたら終了
            if flag == False:
                break

            # 画像の縦横を指定サイズに変形
            img = Image.fromarray(frame)
            img = img.resize((width, im_height),Image.ANTIALIAS)

            # 黒く塗りつぶす用の背景画像を作成
            bg = Image.new("RGB",[width,height],(0,0,0))

            # 元の画像を、背景画像のセンターに配置
            bg.paste(img,(int((width-img.size[0])/2),int((height-img.size[1])/2)))

            # opencv用に変換
            out_frame = np.asarray(bg)

            output_frames.append(out_frame)

        print ("元動画読み込み終了")

        # 補間後のフレーム
        interpolarted_frames = []
        # フレーム補間用比率
        fps_interpolation = fps / 30

        print("補間動画生成開始")
        cnt = 0

        # 最後の１つ手前（補間ができる状態）までループ
        # for _ in tqdm(range(round(count * (30 / fps)) - 1)):
        for _ in range(round(count * (30 / fps)) - 1):
            # 補間した出力CNT
            inter_cnt = cnt * fps_interpolation
            # INDEXと比率（整数部と小数部）
            inter_cnt_rate, inter_cnt_idx = math.modf(inter_cnt)
            # print("フレーム補間: %s -> %s, idx: %s, rate: %s" % ( cnt, inter_cnt, inter_cnt_idx, inter_cnt_rate ))

            # 前のフレーム
            past_frame = output_frames[int(inter_cnt_idx)]
            # 今回のフレーム
            now_frame = output_frames[int(inter_cnt_idx + 1)]

            # 混ぜ合わせる比率
            past_rate = inter_cnt_rate
            now_rate = 1 - inter_cnt_rate

            # フレーム補間をして出力する
            target_output_frame = cv2.addWeighted(past_frame, past_rate, now_frame, now_rate, 0)

            # # PNG出力
            # cv2.imwrite(interpolation_output_path, target_output_frame.clip(0, 255))

            # # 再読み込み
            # interpolation_img = cv2.imread(interpolation_output_path)

            # 動画出力
            out.write(target_output_frame)

            cnt += 1

        print("補間動画生成終了")

        # 最後にnowを出力
        out.write(now_frame)

        # 終わったら開放
        out.release()
        cap.release()
    except Exception as e:
        print("再エンコード失敗", e)
        print(traceback.format_exc())

    cv2.destroyAllWindows()
    
    print('MMD入力用MP4ファイル再生成', out_path)
    input_video_name = out_name

    # 入力動画ファイル再設定
    input_video = base_path + "/"+ input_video_name
    
    video = cv2.VideoCapture(input_video)
    # 幅
    W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    # 高さ
    H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    # 総フレーム数
    count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # fps
    fps = video.get(cv2.CAP_PROP_FPS)

    print("【再チェック】横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))

!echo "入力映像ファイルは" "$input_video_name" "です。"
!echo ""
!echo "問題なければ、次へ進んで下さい。"

from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)

最後のファイル名が取得できていたら成功です。

---
**【OK】**

ファイルの大きさとfpsが指定通りの場合、入力ファイルをそのまま扱います。

![OK](https://drive.google.com/uc?export=view&id=1lvOhCAj99_NUNDb-wfRAxeu-o0Exth7v)

----
**【再エンコード】**

ファイルの大きさとfpsが指定通りではない場合、再エンコードしたmp4ファイルを入力ファイルとして扱います。

![再エンコード](https://drive.google.com/uc?export=view&id=1xEiy-pdeHWQpt4CLZePg9YT1_7U8bEqz)


## パラメーター設定

パラメーターを設定して下さい。

 - 【O】… Openposeで使用するパラメーター
 - 【M】… 深度推定（mannequinchallenge-vmd）で使用するパラメーター
 - 【V】… VMD生成（VMD-3d-pose-baseline-multi）で使用するパラメーター

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑦】　パラメーター設定

#@markdown ■■■■■■■■■■■■■■■■■■


#@markdown 映像をトレースする際のパラメーターを入力して、セルを実行して下さい。

#@markdown --- 

#@markdown ### 【O】映像に映っている最大人数
#@markdown 映像から取得したい人数を入力して下さい。
#@markdown できるだけこの人数分映っているように、映像データを加工しておいてください。
#@markdown 交差などで多少映ってない分には大体フォローできます。
number_people_max =   1#@param {type: "number"}

#@markdown --- 

#@markdown ### 【O】解析を開始するフレーム
#@markdown 解析を開始するフレームNoを入力して下さい。(0始まり)
#@markdown 最初にロゴが表示されている等、人体が正確にトレースできない場合に、全員が映像内に映っている最初のフレームを指定してください。
frame_first = 0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【M】解析を終了するフレーム
#@markdown 解析を終了するフレームNoを入力して下さい。(0始まり)
#@markdown 【深度推定】で反転や順番を調整する際に、最後まで出力せずとも処理を終了して結果を見ることができます。
#@markdown デフォルト値の「-1」だと、最後まで解析を行います。
end_frame_no = -1  #@param {type: "number"}

#@markdown --- 

#@markdown ### 【M】反転指定リスト
#@markdown Openposeが誤認識して反転しているフレーム番号(0始まり)、人物INDEX順番、反転の内容を指定してください。
#@markdown Openposeが0F目で認識した順番に0, 1, とINDEXが割り当てられます。
#@markdown フォーマット：［＜フレーム番号＞:反転を指定したい人物INDEX,<反転内容>］
#@markdown <反転内容>: R: 全身反転, U: 上半身反転, L: 下半身反転, N: 反転なし
#@markdown 例）[10:1,R]　…　10F目の1番目の人物を全身反転します。
#@markdown message.logに上記フォーマットで、反転出力した場合にその内容を出力しているので、それを参考にしてください。
#@markdown [10:1,R][30:0,U]のように、カッコ単位で複数件指定可能です。
reverse_specific = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【M】順番指定リスト
#@markdown 複数人数トレースで、交差後の人物INDEX順番を指定してください。1人トレースの場合は空欄のままで大丈夫です。
#@markdown Openposeが0F目で認識した順番に0, 1, とINDEXが割り当てられます。
#@markdown フォーマット：［＜フレーム番号＞:0番目に推定された人物のインデックス,1番目に推定された人物のインデックス, …］
#@markdown 例）[10:1,0]　…　10F目は、左から1番目の人物、0番目の人物の順番に並べ替えます。
#@markdown message.logに上記フォーマットで、どのような順番で出力したかを残しているので、それを参考にしてください。
#@markdown [10:1,0][30:0,1]のように、カッコ単位で複数件指定可能です。
#@markdown また、output_XXX.aviでは、推定された順番に人物に色が割り当てられています。体の右半分は赤、左半分は以下の色になります。
#@markdown 0:緑, 1:青, 2:白, 3:黄, 4:桃,  5:水色, 6:濃緑, 7:濃青, 8:灰色, 9:濃黄, 10:濃桃, 11:濃水色
order_specific = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【V】ボーン構造CSVファイル
#@markdown トレース対象モデルのボーン構造CSVファイルのパスを選択もしくは入力して下さい。
#@markdown あにまさ式ミクと、あにまさ式ミク準標準が選べる他、任意のモデルのボーン構造CSVファイルが入力可能です。
#@markdown 任意のモデルボーン構造CSVファイルを入力する場合、Googleドライブの "autotrace" フォルダにcsvファイルをアップロードしてください。
#@markdown そしてcsvファイル名を入力して下さい。
#@markdown [モデルボーン構造CSVファイル出力方法](https://github.com/miu200521358/VMD-3d-pose-baseline-multi/blob/master/born/README.md)
born_model_csv = "born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u6E96\u6A19\u6E96\u30DC\u30FC\u30F3.csv" #@param ["born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u30DC\u30FC\u30F3.csv", "born/\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF\u6E96\u6A19\u6E96\u30DC\u30FC\u30F3.csv"] {allow-input: true}


#@markdown --- 

#@markdown ### 【V】IKで出力するか
#@markdown 足をIKで出力するか、yes か no を選んで下さい。
#@markdown no を入力した場合、FKで出力します
ik_flag = "yes"  #@param ['yes', 'no']
is_ik = 1 if ik_flag == "yes" else 0

#@markdown ---

#@markdown ### 【V】踵位置補正
#@markdown 踵のY軸補正値を数値(小数可)で入力して下さい。
#@markdown マイナス値を入力すると地面に近付き、プラス値を入力すると地面から遠ざかります。
#@markdown ある程度は自動で補正しますが、補正しきれない場合に、設定して下さい。
heel_position = 0.0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】センターZ移動倍率
#@markdown センターZ移動に掛ける倍率を数値(小数可)で入力して下さい。
#@markdown 値が小さいほど、センターZ移動の幅が小さくなります。
#@markdown 0を入力した場合、センターZ軸移動を行いません。
center_z_scale = 1.5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】センターZの滑らかさ
#@markdown センターZの円滑化の度数を指定します
#@markdown 1以上の整数のみを入力して下さい。
#@markdown 度数が大きいほど、円滑化されます。（代わりに動作が小さくなります）
depth_smooth_times = 4  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】滑らかさ
#@markdown モーションの円滑化の度数を指定します
#@markdown 1以上の整数のみを入力して下さい。
#@markdown 度数が大きいほど、円滑化されます。（代わりに動作が小さくなります）
smooth_times = 1  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】移動間引き量
#@markdown 移動キー（IK・センター）の間引きに使用する移動量を数値(小数可)で指定します
#@markdown 指定された範囲内の移動があった場合に間引きされます。
#@markdown 移動間引き量を0にした場合、間引きを行いません。
threshold_pos = 0.5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】回転間引き角度
#@markdown 回転キーの間引きに使用する角度(0～180度まで小数可)を指定します
#@markdown 指定された角度以内の回転があった場合に間引きされます。
threshold_rot = 5  #@param {type: "number"}

import os
if not os.path.exists("./VMD-3d-pose-baseline-multi/{0}".format(born_model_csv)):
    # 既存のボーン構造CSVでない場合、ドライブの下を参照
    born_model_csv = "/gdrive/My Drive/autotrace/{0}".format(born_model_csv)
    if not os.path.exists(born_model_csv):
        !echo ■■■■■■■■■■■■■■■■
        !echo ■ WARNING
        !echo ■ ボーン構造CSVが見つかりません。ファイル名を確認してください。
        !echo ■ "$born_model_csv"
        !echo ■■■■■■■■■■■■■■■■

!echo 【O】映像に映っている最大人数: "$number_people_max"
!echo 【O】解析を開始するフレーム: "$frame_first"
!echo 【M】解析を終了するフレーム: "$end_frame_no"
!echo 【M】反転指定リスト: "$reverse_specific"
!echo 【M】順番指定リスト: "$order_specific"
!echo 【V】ボーン構造CSVファイル: "$born_model_csv"
!echo 【V】IKで出力するか: "$ik_flag"
!echo 【V】踵位置補正: "$heel_position"
!echo 【V】センターZ移動倍率: "$center_z_scale"
!echo 【V】センターZ滑らかさ: "$depth_smooth_times"
!echo 【V】滑らかさ: "$smooth_times"
!echo 【V】移動間引き量: "$threshold_pos"
!echo 【V】回転間引き角度: "$threshold_rot"

!echo ""
!echo 上記で間違いない場合、次に進んで下さい。


## 自動トレース実行（全実行）

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑧】　自動トレース実行（全実行）

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown フォームの入力がすべて完了したら、このセルを実行してください。
#@markdown 以下の順番で処理が実行されます。

#@markdown 1. Openpose（映像→2D）
#@markdown 2. mannequinchallenge-vmd（深度推定、人物INDEX並び替え）
#@markdown 3. 3d-pose-baseline-vmd（2D→3D）
#@markdown 4. VMD-3d-pose-baseline-multi（3D→VMD）

#@markdown トレース人数にもよりますが、6000Fで大体50～60分くらいかかります
#@markdown Openposeが開始すると、しばらく細長い四角が出たまま動かなくなったように見えます。
#@markdown 再生ボタンの周りがくるくる回っていたら、背後で処理は行われていますので、何も操作せずお待ちください。
#@markdown vmdファイルが生成されていない、pos.txtの中身が空、error.txtだけがある、といった場合は、まずerror.txtの中身を確認して、「エラーが起きた場合」セクションを確認・実行して下さい。

import time
import datetime
import cv2
import shutil
import glob
from google.colab import drive

start_time = time.time()

# 出力フォルダ削除
if os.path.exists("./output"):
    !rm -r ./output

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"
! mkdir -p "$output_json"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo Openpose
! echo ------------------------------------------

# Openpose実行
! cd openpose/ && ./build/examples/openpose/openpose.bin --video "$input_video" --display 0 --model_pose COCO --write_json "$output_json" --write_video "$output_openpose_avi" --frame_first "$frame_first" --number_people_max "$number_people_max"

! echo ------------------------------------------
! echo mannequinchallenge-vmd
! echo ------------------------------------------

! cd mannequinchallenge-vmd && python predict_video.py --video_path "$input_video" --json_path "$output_json" --interval 20 --reverse_specific "$reverse_specific" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max" --end_frame_no "$end_frame_no" --input single_view --batchSize 1
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_specific.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    for i in range(1, number_people_max+1):
        ! echo ------------------------------------------
        ! echo 3d-pose-baseline-vmd ["$i"]
        ! echo ------------------------------------------

        target_name = "_" + now_str + "_idx0" + str(i)
        target_dir = output_json + target_name

        !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

        ! echo ------------------------------------------
        ! echo VMD-3d-pose-baseline-multi ["$i"]
        ! echo ------------------------------------------

        ! cd ./VMD-3d-pose-baseline-multi && python main.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position" -d "$depth_smooth_times"

        # INDEX別結果コピー
        idx_dir_path = drive_dir_path + "/idx0" + str(i)
        ! mkdir -p "$idx_dir_path"
        
        # 日本語対策でpythonコピー
        for f in glob.glob(target_dir +"/*.vmd"):
            shutil.copy(f, idx_dir_path)
        
        ! cp "$target_dir"/pos.txt "$idx_dir_path"
        ! cp "$target_dir"/start_frame.txt "$idx_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■すべての処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"
    ! ls -l "$drive_dir_path"

from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)    

## 自動トレース実行（部分実行）

**「自動トレース実行（全実行）」**を全部終えた後、再度の実行が不要な場合は、これで終了です。

何らかの修正を行いたい場合、必要に応じて、以下方法で再度トレースを行ってください。

1. トレース元動画を変えたい場合
  - 新動画を、**新しいファイル名**で、Googleドライブの**「autotrace」**フォルダにアップロード
  - **「入力映像ファイルアップロード」**のセルを実行
  - **「パラメーター設定」**のセルを実行
  - **「自動トレース実行（全実行）」**のセルを実行
  
2. 【O】のパラメーターを変えたい場合
  - **「パラメーター設定」**の【O】の値を変更
  - **「パラメーター設定」**のセルを実行
  - **「自動トレース実行（全実行）」**のセルを実行

3. 【M】のパラメーターを変えたい場合
  - **「パラメーター設定」**の【M】の値を変更
  - **「パラメーター設定」**のセルを実行
  - **「A) 自動トレース再実行（深度推定）」**のセルを実行
  - 【M】のパラメーターに納得した場合
      -  **「B) 自動トレース再実行（2D→3D）」**のセルを実行
      - **「C) 自動トレース再実行（3D→VMD）」**のセルを実行

4. 【V】のパラメーターを変えたい場合
  - **「パラメーター設定」**の【V】の値を変更
  - **「パラメーター設定」**のセルを実行
  - **「C) 自動トレース再実行（3D→VMD）」**のセルを実行

### A) 自動トレース再実行(深度推定)

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑨-A】　自動トレース再実行(深度推定)

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown パラメーター設定のセルを実行したら、このセルを実行してください。

#@markdown 全実行で深度推定が既に行われていると見なし、人物INDEX並び替え処理以降の処理を行います。

import time
import datetime
import cv2
import os
from google.colab import drive

start_time = time.time()

# 過去深度結果
past_depth_dir_path =  output_json + "_" + now_str + "_depth"

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo mannequinchallenge-vmd
! echo ------------------------------------------
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

! cd mannequinchallenge-vmd && python predict_video.py --video_path "$input_video" --json_path "$output_json" --past_depth_path "$past_depth_dir_path" --interval 20 --reverse_specific "$reverse_specific" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max" --end_frame_no "$end_frame_no" --input single_view --batchSize 1

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_specific.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■【深度推定】の処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"    
    ! ls -l "$drive_dir_path"


from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)    

### B) 自動トレース再実行(2D→3D）

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑨-B】　自動トレース再実行(2D→3D）

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown パラメーター設定のセルを実行したら、このセルを実行してください。

#@markdown 人数分の処理が行われます。

import time
import datetime
import cv2
import os
from google.colab import drive

start_time = time.time()

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

for i in range(1, number_people_max+1):
    ! echo ------------------------------------------
    ! echo 3d-pose-baseline-vmd ["$i"]
    ! echo ------------------------------------------

    target_name = "_" + now_str + "_idx0" + str(i)
    target_dir = output_json + target_name

    !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

    # INDEX別結果コピー
    idx_dir_path = drive_dir_path + "/idx0" + str(i)
    ! mkdir -p "$idx_dir_path"
    ! cp "$target_dir"/pos.txt "$idx_dir_path"

# Googleドライブ再マウント
drive.mount('/gdrive')

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■【2D→3D】の処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo ""
! echo "MMD自動トレース実行結果"

! echo "$drive_dir_path"    
! ls -l "$drive_dir_path"


from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)

### C) 自動トレース再実行(3D→VMD)

In [0]:
#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown 【セル⑨-C】　自動トレース再実行(3D→VMD)

#@markdown ■■■■■■■■■■■■■■■■■■

#@markdown パラメーター設定のセルを実行したら、このセルを実行してください。

#@markdown 人数分の処理が行われます。

import time
import datetime
import cv2
import shutil
import glob
from google.colab import drive

start_time = time.time()

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"


for i in range(1, number_people_max+1):

    ! echo ------------------------------------------
    ! echo VMD-3d-pose-baseline-multi ["$i"]
    ! echo ------------------------------------------
    
    target_name = "_" + now_str + "_idx0" + str(i)
    target_dir = output_json + target_name

    ! cd ./VMD-3d-pose-baseline-multi && python main.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position" -d "$depth_smooth_times"

    # INDEX別結果コピー
    idx_dir_path = drive_dir_path + "/idx0" + str(i)
    ! mkdir -p "$idx_dir_path"
    ! cp "$target_dir"/*.vmd "$idx_dir_path"
    # 日本語対策でpythonコピー
    for f in glob.glob(target_dir +"/*.vmd"):
        shutil.copy(f, idx_dir_path)

# Googleドライブ再マウント
drive.mount('/gdrive')

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■【3D→VMD】の処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo ""
! echo "MMD自動トレース実行結果"

! echo "$drive_dir_path"
! ls -l "$drive_dir_path"


from IPython.display import Audio
Audio("sceneswitch1.mp3", autoplay=True)

# エラーが起きた場合

エラーが起きた場合、vmdファイルが生成されていない場合は、このセクションを上からひとつずつ実行してください。

それでも解決しない場合、導入編の手順に従って、ノートブックのコピーを共有してください。

## 1. Openposeが読み取る最初のフレームに、人数分映っていない場合

error.txtに「最初のフレームに人数分のデータがありません。」と記載されている場合、Openposeが読み取る最初のフレームに人数分のデータがない事が原因です。

下のセルを実行してください。

In [0]:
!find output/json/ -name "*.json" | sort | head -n 1 | xargs ls -l
!find output/json/ -name "*.json" | sort | head -n 1 | xargs cat

【状況】
下記のように、peopleの後ろにデータがない場合、0F目に人物データが取得できていません。

![結果なし](https://drive.google.com/uc?export=view&id=1osssF0NCWply6J0-zPIhN2wm1gT0o6Io)

【解決方法】

人物が映っている最初のフレームを「【O】解析を開始するフレーム」に指定してください。

「Openposeが読み取ったフレームリスト30件」のセルを実行すると、先頭30件のOpenpose結果ファイルが表示されます。

![先頭30件](https://drive.google.com/uc?export=view&id=1lxP78w4NIbQSKWhpfbynCjDKOgmcNp0o)

人物データがないJSONファイルは、ファイルサイズがとても小さいです。（図の場合、0F目が人物データなし）

1人分のデータで大体500Byte前後(0.5KB)のファイルサイズになります。

これを参考にして、先頭のフレーム番号を決めてください。元動画の編集や再アップロードは不要です。

先頭のフレーム番号が決まったら、「パラメーター設定」セクションの「【O】解析を開始するフレーム」に、その番号を入力して、「パラメーター設定」＞「自動トレース実行（全実行）」の順で実行してください。


複数人トレースの場合は、0F目（【O】解析を開始するフレーム）には全員映っている必要があります。
ファイルサイズも人数分増えますので、目安にしてください。

## 2. Openposeが読み取ったフレームリスト先頭30件

In [0]:
ls -l output/json/*.json | head -n 30

## 3. Googleドライブにファイルが追加されない場合

error.txt もvmdも何も出力されなかった場合、セルの出力結果を確認してください。

最後にファイル名のリストが出ている場合、出力自体は成功しています。

ただ、Googleドライブとの連携が済んでいるにも関わらず、データが反映されないケースを確認しています。

詳細は調査中ですが、とりあえずの対応として、クラウド上の元データをダウンロードしてください。

1. 目次の横にある「ファイル」欄をクリックする
2. ヘッダの「更新」をクリックする
3. output ＞ json を開く
4. xxx_depth ＞ output_XXX.avi …　背景AVI(MMD)
5. xxx_idxXX ＞ output_XXX.vmd　…　モーションデータ(MMD)
6. xxx_idxXX ＞ pos.txt　…　3D関節位置データ(Unity)
7. 複数人数のトレースした場合、idxが複数件できています。

![クラウドデータ](https://drive.google.com/uc?export=view&id=1fArRyRdfs1kBLaLTpdkdJ-MYwHNe-UUq)


## 4. すべてやり直したい場合

準備がうまくいかなかった場合などで、全部やり直したい場合、ランタイムをリセットしてください。

ヘッダ　＞　「ランタイム」　＞　「すべてのランタイムをリセット」

![リセット](https://drive.google.com/uc?export=view&id=1AYX8hv5mmAsbhF8o0S3_6hhdNeax9c8J)

確認ダイアログが出ますので、OKして進めてください。

# TIPS

上記の他、思いつくままに、参考になりそうな事を。

## オススメの作業順番


私は以下の順番で作業を行っています。

1. **「【O】映像に映っている最大人数」**に、トレース元動画からトレースしたい人数を入力して、セルを実行する
2. **「自動トレース実行（全実行）」**のセルを実行する
3. 結果がエラーになった場合、「エラーが起きた場合」の**「Openposeが読み取る最初のフレームに、人数分映っているか」**を実行して、人物が映っているフレームを確認する。成功していたら6に移動。
4. **「【O】解析を開始するフレーム」**に、3で見つけたフレーム番号を入力する
5. **「パラメーター設定」**のセルを実行する
6. **「A) 自動トレース再実行(深度推定)」**のセルを実行する
7.   複数人数のトレースで、入れ替わりが認識できていない場合、message.logを見ながら、**「【F】順番指定リスト」**に順番を指定する
8. **「パラメーター設定」**のセルを実行する
9. **A) 自動トレース再実行(深度推定)」**のセルを実行する
10. 入れ替わりの順番指定が納得いくまで、7～9を繰り返す
11. 意図しない回転があった場合、message.logを見ながら、**「【F】反転指定リスト」**に該当フレームの正しい反転状況を指定して設定する　
  - ※副薄人数トレースで並び順番が変わると、反転指定の人物INDEXも変わるため、順番指定が終わった後に実行した方がよい。
12. **「パラメーター設定」**のセルを実行する
13. **A) 自動トレース再実行(深度推定)」**のセルを実行する
14. 納得いくまで、11～13を繰り返す
15. 入れ替えや回転の指定が完了したら、**「B) 自動トレース再実行(2D→3D）」**を実行する
16. **「パラメーター設定」**の【V】の値を調整する
17. **「パラメーター設定」**のセルを実行する
18. **「C) 自動トレース再実行(3D→VMD)」**のセルを実行する。複数人数のトレースの場合も１回で人数分出力されます。
19. 納得いくまで、15～18を繰り返す

頑張って下さい！

## トレースしやすい動画

 - 固定カメラである
 - 関節がはっきり分かる
    - ロングスカートや和服等、関節が見えにくい人体は苦手です
    - 背景が人物と似た色である、影が濃く映っている、などの場合、トレースを間違える事が多いです
    - 手首・足首まで映っている方が、精度が高くなります
    - 真っ黒なズボン等で、左右の区別がつきにくい場合、精度が落ちます
 - 最初のフレームで前向きである
   - 後ろや横を向いていると始めのデータが綺麗に取れません（一度正面を向いてくれると直る事が多いです）
 - 最初のフレームで全身の関節が判別できる
   - どこか隠れていると、それだけ精度が落ちます
 - 頭が上、足が下
   - 逆立ちやキックなどで高く足が上がっている場合、足を手と誤認識します。（特に首より足の根元の関節が上になっている場合など）


## 課題

 - 回転が取れていない関節
   - 手首
   - 指


# ライセンス

MMD自動トレースの結果を公開・配布する場合は、必ずライセンスのご確認をお願い致します。Unityの場合も同様です。

ライセンスを記載いただけたらとても有難いです。

[MMDモーショントレース自動化キットライセンス](https://ch.nicovideo.jp/miu200521358/blomaga/ar1686913)